In [1]:
#Import pandas and SQLAlchemy
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Load the ratings file from Good Books and import it into a Dataframe
good_books_rating = "BX-Book-Ratings.csv"
good_books_rating_df = pd.read_csv(good_books_rating,encoding='latin1',sep=';' , error_bad_lines=False)
good_books_rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [3]:
#Load the data file from Book Crossing and import it into a Dataframe
book_crossing_data = "books.csv"
book_crossing_data_df = pd.read_csv(book_crossing_data)
book_crossing_data_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
#Load the ratings file from Book Crossing and import it into a Dataframe
book_crossing_ratings = "ratings.csv"
book_crossing_ratings_df = pd.read_csv(book_crossing_ratings)
book_crossing_ratings_df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [5]:
#Transform the Good Books ratings DF

#We rename the columns to make them uniform with the table
good_books_rating_transformed_df = good_books_rating_df.rename(columns={
    "ISBN" : "isbn",
    "Book-Rating" : "book_rating"
})

#Removing the user id column
good_books_rating_transformed_df = good_books_rating_transformed_df.drop(columns=['User-ID'])

#Dropping rows without isbn numbers
good_books_rating_transformed_df = good_books_rating_transformed_df.dropna()

#Dropping duplicates to ensure our primary key is unique
good_books_rating_transformed_df = good_books_rating_transformed_df.drop_duplicates(['isbn'])

#setting the index to isbn
good_books_rating_transformed_df = good_books_rating_transformed_df.set_index('isbn')

good_books_rating_transformed_df.head()

,book_rating
isbn,
034545104X,0
0155061224,5
0446520802,0
052165615X,3
0521795028,6


In [6]:
#Transform the Book Crossing data DF

book_crossing_data_transformed_df = book_crossing_data_df[["book_id", "isbn", "authors", "original_publication_year", "original_title"]]

#Dropping NA's to ensure our ISBN will be unique
book_crossing_data_transformed_df = book_crossing_data_transformed_df.dropna()

#Setting ISBN as the index
book_crossing_data_transformed_df= book_crossing_data_transformed_df.set_index('isbn')


book_crossing_data_transformed_df.head()


,book_id,authors,original_publication_year,original_title
isbn,,,,
439023483,2767052,Suzanne Collins,2008.0,The Hunger Games
439554934,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone
316015849,41865,Stephenie Meyer,2005.0,Twilight
61120081,2657,Harper Lee,1960.0,To Kill a Mockingbird
743273567,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby


In [7]:
#Transform the Book Crossing ratings DF
book_crossing_ratings_transformed_df = book_crossing_ratings_df.rename(columns={
    "rating" : "book_rating"
})

#Dropping duplicates of the combination of user_id and book_id and removed any subsequent duplicates
book_crossing_ratings_transformed_df = book_crossing_ratings_transformed_df.drop_duplicates(subset=["user_id", "book_id"], keep='first')

#Set the index as user id
book_crossing_ratings_transformed_df = book_crossing_ratings_transformed_df.set_index('user_id')

book_crossing_ratings_transformed_df.head()

,book_id,book_rating
user_id,,
314,1,5
439,1,3
588,1,5
1169,1,4
1185,1,4


In [8]:
#At this point you will need to leave this notebook for a time and create a database in PostGres named "bookrating_db"

#Then create the tables using the schema below

###
create table good_books_ratings(
	isbn text primary key,
	book_rating int 
);

create table book_crossing_ratings(
	user_id int,
	book_id int,
	book_rating int,
	primary key (user_id, book_id)
);

create table book_crossing_data(
	isbn text primary key,
	book_id int,
	authors text,
	original_publication_year int,
	original_title text
);

In [9]:
#Creating the database connection
#Switch our your password where marked below.
connection_string = "postgres:[PASSWORDHERE]@localhost:5432/bookrating_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# Confirm tables
engine.table_names()

['book_crossing_data', 'good_books_ratings', 'book_crossing_ratings']

In [11]:
#Loading the data from the database into DataFrames
book_crossing_data_transformed_df.to_sql(name='book_crossing_data', con=engine, if_exists='append', index=True)

In [12]:
book_crossing_ratings_transformed_df.to_sql(name='book_crossing_ratings', con=engine, if_exists='append', index=True)

In [13]:
good_books_rating_transformed_df.to_sql(name='good_books_ratings', con=engine, if_exists='append', index=True)